In [34]:
# Данный код является парсером, который собирает информацию с сайта по продаже игр и сохраняет ее в локальную БД
# собирается информация с названием интересующей игры, изначальной стоимостью и нынешней стоимостью
# написан код, чтобы проследить падение или рост цен на избранные пользователем игры
# основное внимание уделяется столбцу price_now, тут мы со временем можем наблюдать падение цены. А столбец price_first присутствует для проверки на возможную манипуляцию со стороны продавца

# !!!ВАЖНО!!! после запуска код работает от 21 до 60 секунд (при условии, что выбраны дефолтные 6 ссылок)
# это замедление внедрено для симуляции действий пользователя проверяющего интересующие игры, чтобы действия программы не выглядели ботом

from datetime import datetime
from bs4 import BeautifulSoup
import time
import requests
import sqlite3
import random

class Parser:
    # этот класс создан с целью открыть страницу, вытащить название и цены интересующего продукта
    def __init__(self, url):
        self.url = url

    # метод отвечает за парсинг самой страницы и собирает весь ее текс
    def open_html(self):
        html = requests.get(self.url)
        return html.text

    # метод собирает название продукта
    def name_parser(self, html):
        soup = BeautifulSoup(html, 'html.parser')

        name_element = soup.find_all('h1', class_='product_title entry-title elementor-heading-title elementor-size-default' )
        name_element = name_element[0].text.strip('Купить (Версия для РФ)')
        return name_element

    # метод собирает первоначальную цену и текущую цену
    def price_parser(self, html):
        soup = BeautifulSoup(html, 'html.parser')

        price_elements = soup.find_all('span', class_='screen-reader-text')

        price_elements[0] = price_elements[0].text.strip('Первоначальная цена составляла \xa0₽.')
        price_elements[1] = price_elements[1].text.strip('Текущая цена: \xa0₽.')
        price_elements.pop(2)
        return price_elements

class DatabaseManager:
    # этот класс подключается, создает таблицу в БД и сохраняет собранные парсером данные
    def __init__(self):
        self.connection = sqlite3.connect('DB_for_parsing.db')
        self.cursor = self.connection.cursor()
        self.create_table()

    # метод создает таблицу в БД, если таковой еще не существует
    def create_table(self):
        IGM_prices = '''
        create table if not exists wanted_games (
            name varchar(50),
            price_first varchar(30),
            price_now varchar(30),
            date varchar(30)
        )
        '''
        self.cursor.execute(IGM_prices)
        self.connection.commit()

    # метод сохраняет собранные данные в созданную таблицу по имени и ценам
    def insert_name_prices(self, name, price_first, price_now):
        date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        sql_insert = '''
        insert into wanted_games (name, price_first, price_now, date)
        values (?, ?, ?, ?)
        '''
        self.cursor.execute(sql_insert, (name, price_first, price_now, date))
        self.connection.commit()

    # закрываем соединение с БД
    def close(self):
        self.cursor.close()
        self.connection.close()


def main_parser_engin():
    # эта функция запускает ранее созданные классы в работу, подавая на вход выбранные url и путь к БД

    # ниже идет присвоение url переменным с понятным названием, для упрощения редактирования списка urls
    mortal_kombat_11 = 'https://igm.gg/shop/games/mortal-kombat-11/'
    manor_lords = 'https://igm.gg/shop/games/manor-lords/'
    resident_evil_2 = 'https://igm.gg/shop/games/resident-evil-2/'
    helldivers_2 = 'https://igm.gg/shop/games/helldivers-2-2-3/'
    metro_exodus = 'https://igm.gg/shop/games/metro-exodus-gold/'
    pacific_drive = 'https://igm.gg/shop/games/pacific-drive/'

    urls = [helldivers_2, metro_exodus, pacific_drive, resident_evil_2, manor_lords, mortal_kombat_11]
    db_path = r'C:\Users\Роман\Desktop\BD\IGM_favorite_games_prices.db'

    # этот цикл работает со списком url созданным выше
    # каждый url проходит через парсер и необходимая информация сохраняется в БД
    # в конце цикла есть переменная delay, которая играет роль задержки перед запуском новой итераци, дабы сайт не воспринял нашу программу как бота
    for url in urls:
        parser = Parser(url)
        db_manager = DatabaseManager()

        try:
            html = parser.open_html()
            prices = parser.price_parser(html)
            name = parser.name_parser(html)
            db_manager.insert_name_prices(name, prices[0], prices[1])
            print('Данные сохранены')
        except Exception as e:
            print(f'Ошибка: {e}')
        finally:
            db_manager.close()

        delay = random.uniform(3, 10)
        time.sleep(delay)
    print('Программа завершила работу') 


main_parser_engin()
# Дождитесь окончания программы (появится сообщение "Программа завершила работу")

Данные сохранены
Данные сохранены
Данные сохранены
Данные сохранены
Данные сохранены
Данные сохранены
Программа завершила работу


In [35]:
# можно запустить этот код, посмотреть, что появилось в базе данных
sqlite_connection = sqlite3.connect('DB_for_parsing.db')
cursor = sqlite_connection.cursor()

cursor.execute("select * from wanted_games")
cursor.fetchall()

[('HELLDIVERS 2', '4 699', '3 899', '2024-07-29 00:34:44'),
 ('Metro Exodus – Gold Edition', '1 599', '599', '2024-07-29 00:34:51'),
 ('Pacific Drive', '1 799', '1 699', '2024-07-29 00:34:56'),
 ('Resident Evil 2', '2 999', '799', '2024-07-29 00:35:06'),
 ('Manor Lords', '1 499', '1 199', '2024-07-29 00:35:10'),
 ('Mortal Kombat 11', '4 399', '399', '2024-07-29 00:35:17'),
 ('HELLDIVERS 2', '4 699', '3 899', '2024-07-29 00:36:30'),
 ('Metro Exodus – Gold Edition', '1 599', '599', '2024-07-29 00:36:38'),
 ('Pacific Drive', '1 799', '1 699', '2024-07-29 00:36:47'),
 ('Resident Evil 2', '2 999', '799', '2024-07-29 00:36:52'),
 ('Manor Lords', '1 499', '1 199', '2024-07-29 00:37:02'),
 ('Mortal Kombat 11', '4 399', '399', '2024-07-29 00:37:10'),
 ('HELLDIVERS 2', '4 699', '3 899', '2024-07-29 00:46:14'),
 ('Metro Exodus – Gold Edition', '1 599', '599', '2024-07-29 00:46:18'),
 ('Pacific Drive', '1 799', '1 699', '2024-07-29 00:46:27'),
 ('Resident Evil 2', '2 999', '799', '2024-07-29 00:46

In [36]:
# так можно выделить нужный столбец с ценой на определенный товар (в данном случае Мортал Комбат)
cursor.execute('select price_now from wanted_games where name = "Mortal Kombat 11"')
cursor.fetchall()

[('399',), ('399',), ('399',), ('399',)]

In [37]:
cursor.close()